In [4]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.utils import resample

from supervised.automl import AutoML

In [2]:
df_init = pd.read_csv('data_v1.csv', sep=';')
df_init.head()

,Id,Number,Result,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_106,...,Feature_99,Feature_100,Feature_111,Feature_1122,Feature_112,Feature_113,Feature_114,Feature_115,Feature_116,Feature_117
0,1,10359/C2020,2,2,56,12,1,7.0,4,2,...,0.0,0.0,1,0,1,1,1.0,1,1,0
1,2,10346/C2020,2,2,69,19,1,6.0,4,2,...,0.0,0.0,1,0,1,1,1.0,0,1,1
2,3,10311/C2020,2,1,66,8,1,4.0,4,2,...,0.0,0.0,1,0,1,1,1.0,0,1,0
3,4,10292/C2020,2,2,62,16,1,NaN,3,2,...,0.0,0.0,1,0,1,1,0.0,0,1,1
4,5,10283/C2020,2,2,67,30,1,NaN,4,2,...,0.0,0.0,1,0,1,1,1.0,0,1,0


In [7]:
target_feature = 'Result'
features = ['Feature_3', 'Feature_34', 'Feature_30', 'Feature_63', 'Feature_33', 'Feature_31', 'Feature_6', 'Feature_41', 'Feature_46', 'Feature_51', 'Feature_47']

df_ = df_init[[target_feature] + features]

num_features = ['Feature_2', 'Feature_3', 'Feature_33', 'Feature_34', 'Feature_36', 'Feature_37', 'Feature_38', 'Feature_39', 'Feature_40', 
                'Feature_41', 'Feature_42', 'Feature_43', 'Feature_44', 'Feature_45', 'Feature_46', 'Feature_47', 
                'Feature_48', 'Feature_49', 'Feature_50', 'Feature_51', 'Feature_53', 'Feature_55', 'Feature_57', 
                'Feature_58', 'Feature_59', 'Feature_64', 'Feature_70', 'Feature_71', 'Feature_72', 'Feature_73']

special_features = ['Feature_17', 'Feature_18', 'Feature_23', 'Feature_74', 'Feature_75', 'Feature_76', 'Feature_77', 
                    'Feature_78', 'Feature_79', 'Feature_80', 'Feature_81', 'Feature_82', 'Feature_83', 'Feature_84', 
                    'Feature_85', 'Feature_86', 'Feature_87', 'Feature_88', 'Feature_89', 'Feature_90', 'Feature_91', 
                    'Feature_92', 'Feature_93', 'Feature_94', 'Feature_95', 'Feature_96', 'Feature_97', 'Feature_98', 
                    'Feature_99', 'Feature_100']

for col in special_features:
    if col in df_.columns:
        df_[col].fillna(0, inplace=True)

cat_features = []
for col in df_.drop(target_feature, axis=1).columns:
    if col in num_features:
        df_[col].fillna(df_[col].median(), inplace=True)
        df_[col] = df_[col].astype('float64')
    else:
        cat_features.append(col)
        df_[col].fillna(-1, inplace=True)
        df_[col] = df_[col].astype('int64')

# Result=0 - alive, Result=1 died
df_.loc[df_[target_feature] == 1, target_feature] = 0
df_.loc[df_[target_feature] == 2, target_feature] = 1

df_train, df_test = train_test_split(
    df_,
    shuffle=True,
    test_size=0.25,
    random_state=0,
    stratify=df_[target_feature],
)

# Separate majority and minority classes
df_majority = df_train[df_train[target_feature] == 1]
df_minority = df_train[df_train[target_feature] == 0]

# Upsample majority class
df_majority_upsampled = resample(
    df_majority,
    replace=True,
    n_samples=100,
    random_state=0,
)
# Upsample minority class
df_minority_upsampled = resample(
    df_minority,
    replace=True,
    n_samples=100,
    random_state=0,
)
# Combine minority class with downsampled majority class
df_train_sampled = pd.concat([df_majority_upsampled, df_minority_upsampled])

X_train = df_train_sampled[features]
y_train = df_train_sampled[target_feature]

X_test = df_test[features]
y_test = df_test[target_feature]

automl = AutoML()
automl.fit(X_train, y_train)

predictions = automl.predict(X_test)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a D

AutoML directory: AutoML_1
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Baseline', 'Linear', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble availabe models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 3 models
1_Baseline logloss 0.693147 trained in 2.81 seconds
2_DecisionTree logloss 0.809512 trained in 35.16 seconds
3_Linear logloss 0.154981 trained in 6.11 seconds
* Step default_algorithms will try to check up to 3 models


ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


4_Default_Xgboost logloss 0.161967 trained in 7.83 seconds
5_Default_NeuralNetwork logloss 0.490609 trained in 1.68 seconds
6_Default_RandomForest logloss 0.282351 trained in 9.13 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.129958 trained in 0.85 seconds


An input array is constant; the correlation coefficient is not defined.


AutoML fit time: 86.48 seconds
AutoML best model: Ensemble


ntree_limit is deprecated, use `iteration_range` or model slicing instead.
